In [1]:
import torch
import torch.nn as nn
import numpy as np
from datasets import load_dataset
from string import punctuation
from collections import Counter

In [1]:
#!pip install --upgrade datasets
dataset = load_dataset("cnn_dailymail","3.0.0")

NameError: name 'load_dataset' is not defined

In [2]:
len(dataset['train']), len(dataset['test']), len(dataset['validation'])

NameError: name 'dataset' is not defined

In [4]:
train_articles = np.array(dataset['train'][:14000]['article'])
train_summaries = np.array(dataset['train'][:14000]['highlights'])
text_training = {'articles':train_articles,
                 'summaries':train_summaries}

validation_articles = np.array(dataset['validation'][:]['article'])
validation_summaries = np.array(dataset['validation'][:]['highlights'])
text_validation = {'articles':validation_articles,
                   'summaries':validation_summaries}

test_articles = np.array(dataset['test'][:]['article'])
test_summaries = np.array(dataset['test'][:]['highlights'])
text_test = {'articles':test_articles,
             'summaries':test_summaries}

#Pre-Processing

In [5]:
def removing_unwanted_characters(text):
    punctuations = punctuation.replace("'","")
    punctuations = punctuation.replace(".","")
    punctuations = punctuation.replace(",","")
    punctuations += '•1234567890'

    text = "".join([c for c in text if c not in punctuations])
    text = text.split("\n")
    text = " ".join(text)
    text = text.strip()
    text = " ".join(text.split())

    new_text = str(text[0])
    for char in range(1, len(text)):
        if (text[char] == '.' or text[char] == ',') and new_text[-1] != ' ':
            new_text += ' '
            new_text += text[char]
        elif (new_text[-1] == '.' or new_text[-1] == ',') and text[char] != ' ':
            new_text += ' '
            new_text += text[char]
        else:
            new_text += text[char]

    return new_text

In [6]:
def tokenizing(data):
    all_unique_words = []
    for text in data:
        all_unique_words.extend(text.split())
    vocab = set(all_unique_words)
    vocab = {word:ii for ii, word in enumerate(vocab, 3)}# '<pad>': 0, '<bos>': 1, '<eos>': 2

    didicted_tokens = ['<pad>', '<bos>', '<eos>']
    for i in range(3):
        vocab[didicted_tokens[i]] = i

    return vocab

In [7]:
def convert_words_to_tokens(data, vocab):
    data_ints = []

    for text in data:
        data_ints.append([vocab.get(word) for word in text.split() if vocab.get(word) != None])
        data_ints[-1].insert(0,1)
        data_ints[-1].append(2)

    size = int(data.shape[0] // 2)
    input_data = data_ints[:size]
    target_data = data_ints[size:]

    return input_data, target_data

In [8]:
def remove_short_articles(data, input_data, target_data):
    small_lenghts_indices = [index for index, text in enumerate(input_data) if len(text) < 100]
    small_lenghts_indices = small_lenghts_indices[::-1]
    for index in small_lenghts_indices:
         input_data.pop(index)
         target_data.pop(index)

    size = int(data.shape[0] // 2)
    small_lenghts_indices_targets = [length + size for length in small_lenghts_indices]
    small_lenghts_indices = small_lenghts_indices + small_lenghts_indices_targets
    data = np.delete(data, small_lenghts_indices, axis=0)

    return data, input_data, target_data

In [9]:
def paddings(input_data, length):
    size = len(input_data)
    for i in range(size):
        if len(input_data[i]) < length:
            rem = length - len(input_data[i])
            input_data[i].extend([0]*rem)

        elif len(input_data[i]) > length:
            input_data[i] = input_data[i][:length]

    return input_data

In [10]:
def preprocessing(data, length_data, length_target, vocab):
    keys = list(data.keys())
    data = np.concatenate((data.get(keys[0]), data.get(keys[1])))
    data = np.array([text.lower() for text in data])
    items = data.shape[0]

    for i in range(items):
       data[i] += " ."
       data[i] = removing_unwanted_characters(data[i])
    if not vocab:
        vocab = tokenizing(data)

    input_data, target_data = convert_words_to_tokens(data, vocab)

    data, input_data, target_data = remove_short_articles(data, input_data, target_data)

    input_data = paddings(input_data, length_data)
    target_data = paddings(target_data, length_target)

    return data, torch.tensor(input_data), torch.tensor(target_data), vocab

In [11]:
text_train, train_data, train_target, vocab = preprocessing(text_training, 1000, 85, {})

In [12]:
text_val, val_data, val_target, vocab = preprocessing(text_validation, 1000, 85, vocab)

In [13]:
text_test, test_data, test_target, vocab = preprocessing(text_test, 1000, 85, vocab)

In [14]:
type(train_data), type(train_target), type(vocab)

(torch.Tensor, torch.Tensor, dict)

In [15]:
type(val_data), type(val_target), type(vocab)

(torch.Tensor, torch.Tensor, dict)

In [16]:
type(test_data), type(test_target), type(vocab)

(torch.Tensor, torch.Tensor, dict)

In [17]:
train_data.shape, train_target.shape, len(vocab)

(torch.Size([13966, 1000]), torch.Size([13966, 85]), 113285)

In [18]:
val_data.shape, val_target.shape, len(vocab)

(torch.Size([13357, 1000]), torch.Size([13357, 85]), 113285)

In [19]:
test_data.shape, test_target.shape, len(vocab)

(torch.Size([11473, 1000]), torch.Size([11473, 85]), 113285)

In [20]:
embedding_dim = 256
vocab_size = len(vocab)
embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=vocab['<pad>'])

In [21]:
train_data = train_data.cuda()
embedding_layer = embedding_layer.cuda()

In [25]:
training_data = embedding_layer(train_data)
validating_data = embedding_layer(val_data)
testing_data = embedding_layer(test_data)

OutOfMemoryError: CUDA out of memory. Tried to allocate 13.32 GiB. GPU 0 has a total capacity of 14.74 GiB of which 1.09 GiB is free. Process 5998 has 13.65 GiB memory in use. Of the allocated memory 13.53 GiB is allocated by PyTorch, and 4.15 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [26]:
training_data.shape, validating_data.shape, testing_data.shape

NameError: name 'training_data' is not defined

In [52]:
word_vector = embedding_layer.weight

In [68]:
word_vectors = word_vector.clone().detach()

In [73]:
words = {'vocab':vocab,
         'word_vectors':word_vectors}

torch.save(words, 'words.pt')

#Same for validation and testing data

In [79]:
text_data, val_data, val_target, vocab = preprocessing(text, 1000, 85, vocab)

In [80]:
text_data.shape, val_data.shape, val_target.shape

((27932,), torch.Size([13966, 1000]), torch.Size([13966, 85]))